In [ ]:
from google.cloud import pubsub_v1
from google.oauth2 import service_account

# set this to your initials
project_id = "welu420"
topic_id = "load_new_rides"

# the downloaded key filename which you uploaded to this Colab session
key_path = "welu420-8af84caa5bcf.json.json"

# create credentials
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

# create publisher and topic objects
publisher = pubsub_v1.PublisherClient(credentials=credentials)
topic_path = publisher.topic_path(project_id, topic_id)

data = "Message from Python"
data = data.encode("utf-8") # data must be a bytestring
    
future = publisher.publish(topic_path, data)
print(f"Published message with ID {future.result()}")


In [ ]:
subscription_id = "rental_subscription"
timeout = 50.0  # number of seconds the subscriber should listen for messages

subscriber = pubsub_v1.SubscriberClient(credentials=credentials)
subscription_path = subscriber.subscription_path(project_id, subscription_id)

def callback(message):
    """ This is the method which is called when a message arrives """
    print("Received message: {}".format(message))
    message.ack()  # we acknowledge the message

# start the subscription
streaming_pull_future = subscriber.subscribe(subscription_path, callback=callback)
print("Listening for messages on {}..\n".format(subscription_path))

# wrap subscriber in a 'with' block to automatically call close() when done
with subscriber:
    try:
        streaming_pull_future.result(timeout=timeout)
    except TimeoutError:
        streaming_pull_future.cancel()
